In [1]:
!pip install segmentation_models_pytorch -q

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.3/121.3 kB 8.1 MB/s eta 0:00:00


In [2]:
import os
from tqdm.notebook import tqdm
import gc
from torch.nn import Parameter
import torch.nn.functional as F
import torch.nn as nn
import math
import timm
import pandas as pl
import torch
import numpy as np
from torch.amp import GradScaler
import cv2
import random
from tqdm.notebook import tqdm
from torch.autograd import Variable
from skimage.metrics import structural_similarity as ssim
import pandas as pd
import segmentation_models_pytorch as smp
import matplotlib.pyplot as plt
from PIL import Image
from scipy import ndimage

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(228)

In [4]:
train_msk = np.load('/kaggle/input/ioai-2025-preparation-class-lesson-8-homework/msk_array.npy')
train_images = sorted(os.listdir('/kaggle/input/ioai-2025-preparation-class-lesson-8-homework/data/train'))
test_images = sorted(os.listdir('/kaggle/input/ioai-2025-preparation-class-lesson-8-homework/data/test'))
test_msk = np.zeros((len(test_images), train_msk.shape[1], train_msk.shape[2]))

train_images = [f'/kaggle/input/ioai-2025-preparation-class-lesson-8-homework/data/train/{path}' for path in train_images]
test_images = [f'/kaggle/input/ioai-2025-preparation-class-lesson-8-homework/data/test/{path}' for path in test_images]

In [5]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, path_image, msks):
        self.path_image = path_image
        self.msks = msks
        self.image_size = 224  # Задаем размер изображений

    def __len__(self):
        return len(self.path_image)

    def __getitem__(self, i):
        img = cv2.imread(self.path_image[i])
        msk = self.msks[i]
        msk = ndimage.shift(msk, (-25,-25))
        msk = msk[:, :, None]

        # Нормализация изображения
        img = (img / 255.)
        img = np.transpose(img, (2, 0, 1)).astype(np.float32)
        img = torch.from_numpy(img)
        msk = torch.from_numpy(msk)

        return img, msk

In [6]:
Dataset(train_images, train_msk).__getitem__(1)[0].shape

torch.Size([3, 640, 640])

In [7]:
Dataset(train_images, train_msk).__getitem__(1)[1].shape

torch.Size([640, 640, 1])

In [8]:
def dice_coeff(pred, target, eps=1e-6, treshold=0.5):
    pred = (pred > treshold).float()  # Бинаризуем предсказания
    target = target.float()  # Убеждаемся, что целевые данные в формате float

    intersection = torch.sum(pred * target)
    union = torch.sum(pred) + torch.sum(target)
    
    dice = (2. * intersection + eps) / (union + eps)
    return dice

In [9]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.unet = smp.UnetPlusPlus(
            'tu-tf_efficientnetv2_l.in21k_ft_in1k',
            encoder_weights='imagenet',
            classes=1,
            decoder_channels=[256, 128, 64, 32, 16]
        )

    def forward(self, x):
        return self.unet(x)

In [10]:
gc.collect()
torch.cuda.empty_cache()

# Гиперпараметры
batch_size = 4
valid_batch_size = 4
epochs = 10
lr = 3.22e-4
clip_grad_norm = 15.28
DEVICE = 'cuda'
params_train = {'batch_size': batch_size, 'shuffle': True, 'drop_last': True, 'num_workers': 2}
params_val = {'batch_size': batch_size, 'shuffle': False, 'drop_last': False, 'num_workers': 2}

train_loader = torch.utils.data.DataLoader(Dataset(train_images, train_msk), **params_train)
val_loader = torch.utils.data.DataLoader(Dataset(train_images[:70], train_msk[:70]), **params_val)

# Модель, оптимизатор, планировщик
model = Model().cuda()
num_train_steps = int(len(train_loader) / batch_size * epochs)
loss_func = smp.losses.DiceLoss(mode="binary", smooth=1.)
scaler = GradScaler()
optimizer = torch.optim.AdamW(model.parameters(), lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, len(train_loader) * epochs, 1e-6)

# Обучение
for epoch in range(epochs):
    model.train()
    average_loss = 0
    tk0 = tqdm(enumerate(train_loader), total=len(train_loader))
    
    # Тренировочный цикл
    for batch_number, (img, target) in tk0:
        optimizer.zero_grad()
        img = img.to(DEVICE)
        target = target.to(DEVICE)
        target = target.permute((0, 3, 1, 2))  # Убираем эту строку, если не нужно менять размерность

        with torch.amp.autocast('cuda'):
            outputs = model(img)
            loss = loss_func(outputs, target)

        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip_grad_norm)
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        average_loss += loss.cpu().detach().numpy()
        tk0.set_postfix(loss=average_loss / (batch_number + 1), lr=scheduler.get_last_lr()[0], stage="train", epoch=epoch + 1)

    # Валидация
    model.eval()
    val_loss = 0
    val_dice = 0
    with torch.no_grad():
        for batch_number, (img, target) in tqdm(enumerate(val_loader), total=len(val_loader)):
            img = img.to(DEVICE)
            target = target.to(DEVICE)
            target = target.permute((0, 3, 1, 2))  # Убираем эту строку, если не нужно менять размерность

            with torch.amp.autocast('cuda'):
                outputs = model(img)
                loss = loss_func(outputs, target)

            val_loss += loss.cpu().detach().numpy()
            dice = dice_coeff(outputs.sigmoid(), target)
            val_dice += dice.cpu().detach().numpy()

    avg_val_loss = val_loss / len(val_loader)
    avg_val_dice = val_dice / len(val_loader)
    print(f"Epoch {epoch + 1}/{epochs} - Train Loss: {average_loss / len(train_loader):.4f}, "
          f"Validation Loss: {avg_val_loss:.4f}, Validation Dice: {avg_val_dice:.4f}")

model.safetensors:   0%|          | 0.00/476M [00:00<?, ?B/s]

  0%|          | 0/136 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

Epoch 1/10 - Train Loss: 0.4765, Validation Loss: 0.3129, Validation Dice: 0.7746


  0%|          | 0/136 [00:00<?, ?it/s]

...

  0%|          | 0/18 [00:00<?, ?it/s]

Epoch 9/10 - Train Loss: 0.1144, Validation Loss: 0.0972, Validation Dice: 0.9083


  0%|          | 0/136 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

Epoch 10/10 - Train Loss: 0.1117, Validation Loss: 0.0960, Validation Dice: 0.9092


In [11]:
params_val = {'batch_size': batch_size, 'shuffle': False, 'drop_last': False, 'num_workers': 2}
test_loader = torch.utils.data.DataLoader(Dataset(test_images, test_msk), **params_val)

In [12]:
preds = []
imgs_list = []
target_list = []
model.eval()
average_loss = 0
with torch.no_grad():
    for batch_number,  (img, target)  in enumerate(test_loader):
        img = img.to(DEVICE)
        target = target.to(DEVICE)

        with torch.amp.autocast('cuda'):
            outputs = model(img)

        preds += [outputs.sigmoid().to('cpu').numpy()]

preds = np.concatenate(preds)[:, 0, ...]

In [13]:
preds = (preds > 0.5).astype(np.uint8)

In [14]:
def rle_encode(x, fg_val=1):
    """
    Args:
        x:  numpy array of shape (height, width), 1 - mask, 0 - background
    Returns: run length encoding as list
    """

    dots = np.where(
        x.T.flatten() == fg_val)[0]  # .T sets Fortran order down-then-right
    run_lengths = []
    prev = -2
    for b in dots:
        if b > prev + 1:
            run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths


def list_to_string(x):
    """
    Converts list to a string representation
    Empty list returns '-'
    """
    if x: # non-empty list
        s = str(x).replace("[", "").replace("]", "").replace(",", "")
    else:
        s = '-'
    return s

In [15]:
true_list = [list_to_string(rle_encode(ans)) for ans in preds]

predict_df = pd.DataFrame()
predict_df['Id'] = [f'{x:03d}.jpg' for x in range(150)]
predict_df['Target'] = true_list
predict_df.to_csv('sestra_zaleti_pzh.csv', index = None)